In [1]:
import pandas as pd
import numpy as np

In [8]:
data = pd.read_csv('data/tripadvisor_hotel_reviews.csv')
print(data.describe().round(2))
data.head(5)

         Rating
count  20491.00
mean       3.95
std        1.23
min        1.00
25%        3.00
50%        4.00
75%        5.00
max        5.00


,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5


In [ ]:
# def label(rating, sentiment):
#     if rating > 3:
#         sentiment = 'positive'
#     if rating < 3:
#         sentiment = 'negative'
#     return rating, sentiment

# rating 应该是一个 number in [1~5]，不需要返回
# label 这个函数的功能设计，是给我的 csv data 打上一个 sentiment label。不做其他任何操作。no side effect

In [18]:
# data_label = data['Rating'].label(Rating) 
#  AttributeError: 'Series' object has no attribute 'label'
# 我遇到了这个错误，这个写法不对。我的function return的东西不对，但我不知道要怎么修改

# AttributeError 是说找不到这个属性，所以我不应该这样 “调用（获取属性）”

AttributeError: 'Series' object has no attribute 'label'

In [19]:
def label(rating):
    '''
     params: rating accept number in [1~5] 
     return sentiment like 'positive', 'negative'
    '''
    if rating > 3:
        sentiment = 'positive'
    if rating <= 3:
        sentiment = 'negative'
    return sentiment

label(3)

'negative'

In [24]:
data['label'] = data['Rating'].map(label)

# data['Rating'] is a series (pandas)
# another impletation for map funtion

# label_list = []
# for row in data['Rating']:
#     label_res = label(row)
#     label_list.append(label_res)
# data['label'] = label_list

data.head()

# class data_list:

#     def my_map(self, function):
#         res_list = []
#         for row in data_list:
#             res = function(row)
#             res_list.append(res)
#         return res_list
# data_list.my_map()

,Review,Rating,label
0,nice hotel expensive parking got good deal sta...,4,positive
1,ok nothing special charge diamond member hilto...,2,negative
2,nice rooms not 4* experience hotel monaco seat...,3,negative
3,"unique, great stay, wonderful time hotel monac...",5,positive
4,"great stay great stay, went seahawk game aweso...",5,positive


In [56]:
def positive_count(label):
    count=0
    if label == 'positive':
        count= count + 1
        return count
    return 0

positive_rate = data['label'].map(positive_count).sum()/data['label'].count()
print(f"positive rate is {positive_rate.round(2)}")

positive rate is 0.74


In [51]:
# data['label'].map(positive_count).sum()
data['label'].where(lambda label: label == 'positive').count()

np.int64(15093)

In [57]:
# convector
# split text to words 
def split(text):
    for row in text:

data['Review'].split(' ')
# drop no meaningful words
# 词形还原：
# convert words to vector


AttributeError: 'Series' object has no attribute 'split'

In [58]:
import re
import math
from collections import defaultdict

# 1. 数据准备（简单示例）
data = [
    ("I love this movie", "pos"),
    ("This film was great", "pos"),
    ("What a fantastic experience", "pos"),
    ("I hate this movie", "neg"),
    ("This film was terrible", "neg"),
    ("It was a horrible experience", "neg")
]

# 2. 分词函数（简单英文分词）
def tokenize(text):
    text = text.lower()
    return re.findall(r'\b\w+\b', text)

# 3. 构建词频和类别统计
class NaiveBayesClassifier:
    def __init__(self):# initialization all attributes and dataframe
        self.class_word_counts = defaultdict(lambda: defaultdict(int))  # {class: {word: count}} 用于计数器，默认值是0
        self.class_counts = defaultdict(int)  # {class: total_words_in_class}
        self.doc_counts = defaultdict(int)    # {class: num_docs}
        self.vocab = set() # set是集合，subset子集，superset超集，comlement补集，较于list是去重的
        self.total_docs = 0

    def train(self, data):
        for text, label in data:
            self.total_docs += 1 #因为要计算P（class）先验概率 total_docs = 1
            self.doc_counts[label] += 1 # {'pos':1}
            words = tokenize(text)
            for word in words:
                self.class_word_counts[label][word] += 1
                self.class_counts[label] += 1
                self.vocab.add(word)

# class_word_counts = {'pos' : {'i': 1, 'love': 1, 'this': 1, 'movie': 1}}
#  class_counts = {'pos' : 4,'neg':0}
    # 4. 使用朴素贝叶斯预测
    def predict(self, text):
        words = tokenize(text)
        scores = {}
        for label in self.class_counts:
            # 先验概率 log(P(class))
            prior_prob = math.log(self.doc_counts[label] / self.total_docs) #log 1/1 =0
            cond_prob = 0
            for word in words:
                # 拉普拉斯平滑：P(word|class) = (count + 1) / (total + |V|)
                word_count = self.class_word_counts[label].get(word, 0)
                total_count = self.class_counts[label]
                cond_prob += math.log((word_count + 1) / (total_count + len(self.vocab))) # log (1+1)/8 log是因为连乘是相加
            scores[label] = cond_prob + prior_prob

        # scores = {'pos':0.8, 'neg': 0.3} => max(scores, key=scores.get) => {'pos':0.8}
        return max(scores, key=scores.get)

    # 5. 批量评估准确率
    def evaluate(self, data):
        correct = 0
        for text, label in data:
            pred = self.predict(text)
            if pred == label:
                correct += 1
        return correct / len(data)

# 训练模型
nb = NaiveBayesClassifier()
nb.train(data)

# 评估模型
print("Accuracy:", nb.evaluate(data))

# 测试预测
print("Test:", nb.predict("I love this amazing film"))     # 预测为 pos
print("Test:", nb.predict("Terrible and horrible movie"))  # 预测为 neg


Accuracy: 1.0
Test: pos
Test: neg
